In [3]:
import cv2
import numpy as np
 
img1 = cv2.imread('data/dog.png')
img2 = cv2.imread('data/cat_dog.png')

cv2.imshow('1',img1)
cv2.imshow('2',img2)

img_gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img_gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
 
#创建特征检测器
# sift = cv2.xfeatures2d.SIFT_create()
sift = cv2.SIFT_create()
#计算特征点和描述子
kp1,des1 = sift.detectAndCompute(img_gray1,None)
kp2,des2 = sift.detectAndCompute(img_gray2,None)
 
#创建特征匹配器
index_params = dict(algorithm=1,trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)
 
#对描述子进行特征匹配
matches = flann.knnMatch(des1,des2,k=2) #用的knnmatch匹配
 
 
goods = []   #选择两个匹配对象中好一些的保存下来
for (m,n) in matches:
    if m.distance < n.distance*0.7:
        goods.append(m)
         
print('goods',len(goods))
 
#把找到的匹配特征点保存在goods中，注意单应性矩阵要求最少4个点
if len(goods) >= 4:
    #把goods中的第一幅图和第二幅图的特征点坐标拿出来（坐标要float32且是三维矩阵类型  reshape(-1,1,2)）
    src_points = np.float32([kp1[m.queryIdx].pt for m in goods]).reshape(-1,1,2)
    des_points = np.float32([kp2[m.trainIdx].pt for m in goods]).reshape(-1,1,2)
    print('des_points:',des_points)
     
    #根据匹配上的关键点去计算单应性矩阵
    H,mask = cv2.findHomography(src_points,des_points,cv2.RANSAC,5) #参数5表示：允许有5个关键点的误差
     
    #通过单应性矩阵，计算小图（img1）在大图中的对应位置
    h,w = img1.shape[:2]
    pts = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)  #img1的四个边框顶点，逆时针（坐标从0开始，因此要w-1,h-1）
     
    #用透视变换函数找到这四个顶点对应的img2的位置，不用warpPerspective,这是对图像的透视变换，用perspectiveTransform()
    dst = cv2.perspectiveTransform(pts,H)
    print('pts:',pts)
    print('dst:',dst)
    #在大图中把dst画出来，polylines
    cv2.polylines(img2,[np.int32(dst)],True,[0,0,255],2)
     
else:
    print('matches is not enough')
    exit()
 
res = cv2.drawMatchesKnn(img1,kp1,img2,kp2,[goods],None) #画出匹配的特征点
 
cv2.imshow('res',res)
cv2.waitKey(0)
cv2.destroyAllWindows()

goods 116
des_points: [[[327.06985 127.09097]]

 [[366.79855 131.09135]]

 [[383.10696 132.78703]]

 [[372.89667 137.32654]]

 [[369.9397  139.75502]]

 [[346.9166  147.91455]]

 [[346.9166  147.91455]]

 [[325.558   149.5305 ]]

 [[321.3303  154.09859]]

 [[321.3303  154.09859]]

 [[329.0716  155.39987]]

 [[323.2759  156.9943 ]]

 [[291.65726 157.47958]]

 [[370.18594 162.08487]]

 [[372.49515 164.87234]]

 [[321.21405 125.04322]]

 [[370.9592  172.24402]]

 [[367.33228 172.9473 ]]

 [[366.34933 178.48738]]

 [[361.6843  179.70831]]

 [[403.44427 181.88933]]

 [[489.86398 182.99934]]

 [[489.86398 182.99934]]

 [[331.90796 183.45824]]

 [[456.71774 183.95152]]

 [[343.1976  184.62872]]

 [[325.9569  184.91847]]

 [[481.2005  186.02289]]

 [[290.82858 186.20703]]

 [[450.88614 187.0073 ]]

 [[304.1974  186.93153]]

 [[363.78613 187.52983]]

 [[393.79724 188.32228]]

 [[356.34473 188.37222]]

 [[339.53104 190.07938]]

 [[290.8612  191.59126]]

 [[462.15515 191.70276]]

 [[482.151   193